In [ ]:
!pip install darts

In [ ]:
!pip install prophet

In [ ]:
!pip install lightgbm

In [ ]:
#Optional - Uncomment if using google drive
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
dataset_path = "" #Add path to dataset here "Preprocessed_dataset.csv"

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from darts import TimeSeries
from darts.models import RegressionModel
import lightgbm as lgbm

In [ ]:
def display_results_DARTS(ytest,preds):
    # Plot the daily prediction vs real
    plt.rcParams.update(plt.rcParamsDefault)
    plt.figure(figsize=(12,8))
    ytest.plot()
    preds.plot()
    plt.title("Real vs Prediction")
    plt.legend(['Real','Pred'])
    plt.show()

    #Plot the % difference between real and pred
    ypred=preds.pd_dataframe()
    ytrue=ytest.pd_dataframe()
    r=pd.concat([ypred,ytrue],axis=1)
    r.columns=['Predicted','Real']
    r['Diff']=ypred-ytrue
    r['%diff']=abs((r['Diff'])/r['Real'])*100

    plt.rcParams.update(plt.rcParamsDefault)
    plt.figure(figsize=(12,8))
    plt.title("LGBM Regressor % diff distribution")
    sns.histplot(r['%diff'],bins=list(range(0,51,5)))
    plt.show()

In [ ]:
def display_results_Prophet(test,forecast):
    plt.figure(figsize=(12,8))
    plt.plot(test['y'],label='Real')
    plt.plot(forecast['yhat'],label='Pred')
    plt.fill_between(list(range(len(test))),forecast['yhat_lower'],forecast['yhat_upper'],label='Conf Int',color='orange',alpha=0.5)
    plt.title("Real vs Prediction")
    plt.legend()
    plt.show()

    #Plot the % differences
    rfeats=forecast[['ds','yhat_lower','yhat_upper','yhat']]
    rfeats=pd.concat([rfeats,test.reset_index().y],axis=1)
    rfeats.columns=['Date','Min','Max','Pred','Real']
    rfeats['PredDiff']=abs(rfeats['Real']-rfeats['Pred'])
    rfeats['%diff_Pred']=abs((rfeats['Real']-rfeats['Pred'])/rfeats['Real'])*100

    plt.hist(rfeats['%diff_Pred'],bins=list(range(0,51,5)))
    plt.title('Prophet w Feats % Difference per day')
    plt.xlabel("% difference")
    plt.ylabel("Number of days")
    plt.show()

In [ ]:
df=pd.read_csv(dataset_path)
df